# CSCI 544 - Homework 3
## Word2Vec, Feed Forward Network, Recurrent Neural Network

### Harshitha B R (7530583855)

In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ANLP/HW1/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', error_bad_lines=False)

<ipython-input-3-11bd1a00f613>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ANLP/HW1/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', error_bad_lines=False)
Skipping line 20773: expected 15 fields, saw 22
Skipping line 39834: expected 15 fields, saw 22
Skipping line 52957: expected 15 fields, saw 22
Skipping line 54540: expected 15 fields, saw 22

Skipping line 80276: expected 15 fields, saw 22
Skipping line 96168: expected 15 fields, saw 22
Skipping line 96866: expected 15 fields, saw 22
Skipping line 98175: expected 15 fields, saw 22
Skipping line 112539: expected 15 fields, saw 22
Skipping line 119377: expected 15 fields, saw 22
Skipping line 120065: expected 15 fields, saw 22
Skipping line 124703: expected 15 fields, saw 22

Skipping line 134024: expected 15 fields, saw 22
Skipping line 153938: expected 15 fields, saw 2

In [4]:
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0.0,0.0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2640249,US,53005790,RLI7EI10S7SN0,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,26.0,26.0,N,N,Great value! A must if you hate to carry thing...,I can't live anymore whithout my Palm III. But...,1998-12-07
2640250,US,52188548,R1F3SRK9MHE6A3,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,18.0,18.0,N,N,Attaches the Palm Pilot like an appendage,Although the Palm Pilot is thin and compact it...,1998-11-30
2640251,US,52090046,R23V0C4NRJL8EM,0807865001,307284585,Gods and Heroes of Ancient Greece,Office Products,4,9.0,16.0,N,N,"Excellent information, pictures and stories, I...",This book had a lot of great content without b...,1998-10-15
2640252,US,52503173,R13ZAE1ATEUC1T,1572313188,870359649,Microsoft EXCEL 97/ Visual Basic Step-by-Step ...,Office Products,5,0.0,0.0,N,N,class text,I am teaching a course in Excel and am using t...,1998-08-22


In [5]:
#dropping empty rows
df.dropna(subset=['review_body', 'star_rating'], inplace=True)


In [6]:
import re
import string
def cleanReviews(text):
    if pd.isna(text):
        return ""

    text = text.lower()
    text = text.strip() #to remove any trailing and leading spaces in the review
    text = re.sub(r'<.*?>', '', text) # to remove HTML tags
    text = re.sub(r'[^a-zA-Z\s]', '', text) # to remove special characters, punctuation, and symbols
    text = text.translate(str.maketrans('', '', string.punctuation)) #to remove punctuation

    return text
df['review_body'] = df['review_body'].apply(cleanReviews)

In [7]:
#getting the labels wrt star_rating
df['sentiment'] = df['star_rating'].apply(lambda rating: '1' if int(rating) >= 4 else '0')

In [ ]:
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0.0,0.0,N,Y,Five Stars,Great product.,2015-08-31,1
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31,1
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31,1
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31,0
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2640249,US,53005790,RLI7EI10S7SN0,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,26.0,26.0,N,N,Great value! A must if you hate to carry thing...,I can't live anymore whithout my Palm III. But...,1998-12-07,1
2640250,US,52188548,R1F3SRK9MHE6A3,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,18.0,18.0,N,N,Attaches the Palm Pilot like an appendage,Although the Palm Pilot is thin and compact it...,1998-11-30,1
2640251,US,52090046,R23V0C4NRJL8EM,0807865001,307284585,Gods and Heroes of Ancient Greece,Office Products,4,9.0,16.0,N,N,"Excellent information, pictures and stories, I...",This book had a lot of great content without b...,1998-10-15,1
2640252,US,52503173,R13ZAE1ATEUC1T,1572313188,870359649,Microsoft EXCEL 97/ Visual Basic Step-by-Step ...,Office Products,5,0.0,0.0,N,N,class text,I am teaching a course in Excel and am using t...,1998-08-22,1


In [8]:
#sampling the data and getting 50000 of positive and negetive data
positive_count = (df['sentiment'] == str(1)).sum()
negative_count = (df['sentiment'] == str(0)).sum()
num_samples = min(50000, positive_count, negative_count)

#sample the rows for each sentiment value
positive_samples = df[df['sentiment'] == str(1)].sample(n=num_samples, random_state=11)
negative_samples = df[df['sentiment'] == str(0)].sample(n=num_samples, random_state=11)


In [9]:
#combine the sampled rows into a single DataFrame
sampled_df = pd.concat([positive_samples, negative_samples])

#index of the combined DataFrame is reset
sampled_df.reset_index(drop=True, inplace=True)
sdf = sampled_df

In [14]:
sdf

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,18476273,R2HEJ8GNO1J41R,B00006IRUC,136362376,Epson Premium Photo Paper GLOSSY,Office Products,5,0.0,0.0,N,Y,Epson Premium Photo Paper GLOSSY (8.5x11 Inche...,always the best for my photo printer,2014-11-16,1
1,US,858386,R11DAO5QBXB8MH,B00HYO512C,616641385,iMBAPrice USB 2.0 Printer and Scanner Cable fo...,Office Products,5,0.0,0.0,N,Y,Five Stars,just what i needed,2014-11-06,1
2,US,45996110,R2U6DL71MUODTR,B00LEA8X6S,214113984,Generic 5 Pack (3 Black & 2 Color) Compatible...,Office Products,5,0.0,0.0,N,Y,Five Stars,great savings,2015-02-09,1
3,US,30295038,R3EMH6RFDQPLGX,B00000J19R,354661798,Brother PC-201 Ink Cartridge - Black - Retail ...,Office Products,5,0.0,0.0,N,Y,Five Stars,perfect,2014-07-10,1
4,US,336150,R1T8N0H6VRZI3A,B00CHC5Y3W,406646024,Package of Three Canon P23-DH V Calculator Ink...,Office Products,4,0.0,0.0,N,Y,Four Stars,good product and received it in a timely manner,2015-04-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,US,43528115,R2RL1F9WNWIFV7,B004AGQ8LK,80677933,D@J 4-Pack Full-Set fits Brother LC51 LC 51 In...,Office Products,1,0.0,0.0,N,Y,so I wasted my money,the black cartridge fits other colors are not ...,2015-05-27,0
99996,US,50472709,RQ6SQ0GC6LKAA,B00NWI1ABU,618630895,NEWSTYLE Universal 9-10 Inch Tablet Portfolio ...,Office Products,1,0.0,0.0,N,Y,The logitech keyboard I have is better than th...,im disappointed because i havent had this prod...,2015-04-17,0
99997,US,43123576,R39IE00T0IE8PO,B00NY6OLCK,470651858,E-Z Ink (TM) Compatible Toner TN660 TN630 High...,Office Products,1,14.0,18.0,N,Y,Didn't work at all,price is right but unfortunately it doesnt wor...,2015-07-01,0
99998,US,23942608,R3LQCXOI3IU42L,B00C6OGKNY,44750145,Levenger Circa Leather iPad Foldover Notebook,Office Products,1,0.0,0.0,N,N,I was not satisfied with this binder due to th...,i was not satisfied with this binder due to th...,2014-10-16,0


### Word embeddings

Generating Word2Vec features for
the dataset. Gensim library is used for this purpose.

In [10]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [12]:
for index, word in enumerate(w2v.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(w2v.index_to_key)} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


In [13]:
vec_king = w2v['king']
print(vec_king)

[ 1.25976562e-01  2.97851562e-02  8.60595703e-03  1.39648438e-01
 -2.56347656e-02 -3.61328125e-02  1.11816406e-01 -1.98242188e-01
  5.12695312e-02  3.63281250e-01 -2.42187500e-01 -3.02734375e-01
 -1.77734375e-01 -2.49023438e-02 -1.67968750e-01 -1.69921875e-01
  3.46679688e-02  5.21850586e-03  4.63867188e-02  1.28906250e-01
  1.36718750e-01  1.12792969e-01  5.95703125e-02  1.36718750e-01
  1.01074219e-01 -1.76757812e-01 -2.51953125e-01  5.98144531e-02
  3.41796875e-01 -3.11279297e-02  1.04492188e-01  6.17675781e-02
  1.24511719e-01  4.00390625e-01 -3.22265625e-01  8.39843750e-02
  3.90625000e-02  5.85937500e-03  7.03125000e-02  1.72851562e-01
  1.38671875e-01 -2.31445312e-01  2.83203125e-01  1.42578125e-01
  3.41796875e-01 -2.39257812e-02 -1.09863281e-01  3.32031250e-02
 -5.46875000e-02  1.53198242e-02 -1.62109375e-01  1.58203125e-01
 -2.59765625e-01  2.01416016e-02 -1.63085938e-01  1.35803223e-03
 -1.44531250e-01 -5.68847656e-02  4.29687500e-02 -2.46582031e-02
  1.85546875e-01  4.47265

In [14]:
# to experiment and check w2v
pairs = [
    ('car', 'minivan'),
    ('car', 'bike'),
    ('car', 'airplane'),
    ('car', 'cereal'),
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, w2v.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bike'	0.59
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


In [15]:
result_pretrained1 = w2v.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
result_pretrained2 = w2v.most_similar(positive=['excellent'], negative=['outstanding'], topn=1)

print(f"Pre-trained Model - King - Man + Woman = {result_pretrained1[0][0]}")
print(f"Pre-trained Model - Excellent ~ Outstanding = {result_pretrained2[0][0]}")


Pre-trained Model - King - Man + Woman = queen
Pre-trained Model - Excellent ~ Outstanding = ideal


In [20]:
result_analogy = w2v.most_similar(positive=['pigeon', 'sedan'], negative=['car'], topn=1)

# Print the result
print(f"Car is to Sedan as Pigeon is to {result_analogy[0][0]}")

Car is to Sedan as Pigeon is to falcon


In [34]:
result_analogy = w2v.most_similar(positive=['Summer', 'Cold'], negative=['Hot'])


print("Similarity for 'Summer - Hot + Cold ':", result_analogy[0][0])

Similarity for 'Summer - Hot + Cold ': Winter


In [36]:

result_an = w2v.most_similar(positive=['Japan', 'Pasta'], negative=['Sushi'])
print("Similarity for 'Japan - Sushi + Pasta':", result_an[0][0])


Similarity for 'Japan - Sushi + Pasta': Italy


2b) training our own w2vec model

In [37]:
import gensim.downloader as api
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [38]:
rev = [word_tokenize(review.lower()) for review in tqdm(sdf['review_body'])]

# Define and train our own Word2Vec model - custom training
embedding_size = 300
window_size = 13
min_word_count = 9

w2vModelCustomTraining = Word2Vec(rev, vector_size=embedding_size, window=window_size, min_count=min_word_count, workers=4)


100%|██████████| 100000/100000 [00:24<00:00, 4141.46it/s]


In [39]:
custRes1 = w2vModelCustomTraining.wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
custRes2 = w2vModelCustomTraining.wv.most_similar(positive=['excellent'], negative=['outstanding'], topn=1)

print(f"Pre-trained Model - King - Man + Woman = {custRes1[0][0]}")
print(f"Pre-trained Model - Excellent ~ Outstanding = {custRes2[0][0]}")


Pre-trained Model - King - Man + Woman = et
Pre-trained Model - Excellent ~ Outstanding = good


It can be concluded that the pretrained weights are much better as they have a larger dataset and also, the data has different genre of data in the same. However our data is limited to amazon reviews

3


Here we are using pretrained model to train perceptron and SVM Models. Average vector for each review is taken as the input

We shall give the accuracy using word2Vec as well as tfidf representations

In [40]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import Perceptron

In [41]:
def calculate_average_vector(sentence, model):
    vectors = [model[word] for word in sentence.split() if word in model]
    if not vectors:
        return np.zeros(300, dtype=np.float32)
    return np.mean(vectors, axis=0)


In [42]:
X_word2vec = [calculate_average_vector(review, w2v) for review in sdf['review_body']]
y = sdf['sentiment']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)








In [43]:
perceptron = Perceptron()

perceptron.fit(X_train, y_train)
y_pred = perceptron.predict(X_test)
y_test_str = y_test
y_pred_str = y_pred
f1 = f1_score(y_test_str, y_pred_str, pos_label='1')
precision = precision_score(y_test_str, y_pred_str, pos_label='1')
recall = recall_score(y_test_str, y_pred_str, pos_label='1')
accuracy = accuracy_score(y_test, y_pred)
print("Perceptron Word2Vec ")
print(f"accuracy: {accuracy:.2f} F1-score: {f1:.2f} Precision: {precision:.2f} Recall: {recall:.2f}")

Perceptron Word2Vec 
accuracy: 0.67 F1-score: 0.75 Precision: 0.61 Recall: 0.97


In [44]:
X_text = sdf['review_body']
y = sdf['sentiment']
X_text_train, X_text_test, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42
)
# creating feature extraction of tfidf
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_text_train)
X_test_tfidf = tfidf_vectorizer.transform(X_text_test)

In [46]:
perceptron.fit(X_train_tfidf, y_train)
y_pred_perceptron = perceptron.predict(X_test_tfidf)
y_pred = perceptron.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)

print("Perceptron - TFIDF")
print(f" accuracy: {accuracy:.2f}")

Perceptron - TFIDF
 accuracy: 0.82


In [47]:
from sklearn.svm import LinearSVC

In [48]:
linear_svc_classifier = LinearSVC()

linear_svc_classifier.fit(X_train, y_train)
y_pred = linear_svc_classifier.predict(X_test)
y_test_str = y_test
y_pred_str = y_pred

f1 = f1_score(y_test_str, y_pred_str, pos_label='1')
precision = precision_score(y_test_str, y_pred_str, pos_label='1')
recall = recall_score(y_test_str, y_pred_str, pos_label='1')
accuracy = accuracy_score(y_test_str, y_pred_str)
print("SVM ")
print(f"accuracy: {accuracy:.2f} F1-score: {f1:.2f} Precision: {precision:.2f} Recall: {recall:.2f}")

SVM 
accuracy: 0.81 F1-score: 0.81 Precision: 0.84 Recall: 0.78


In [49]:
linear_svc_classifier.fit(X_train_tfidf, y_train)

y_pred = linear_svc_classifier.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)

print("SVM - TFIDF")
print(f" accuracy: {accuracy:.2f}")

SVM - TFIDF
 accuracy: 0.86


References:
1. https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html

We can notice that TFIDF scores are slightly higher than W2V, maybe since these there are many repetitive words, because of which the frequencies are going to be a deciding factor, hence TFIDF does slightly better

In [50]:
import gc

In [51]:
gc.collect()

110

## task 4

In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import numpy as np
from gensim.models import Word2Vec
import gensim.downloader as api

In [52]:
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)


In [53]:
type(y_train)

numpy.ndarray

In [56]:
y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)

# Converting to PyTorch tensors
XTrainTensor = torch.FloatTensor(X_train)
XTestTensor = torch.FloatTensor(X_test)
yTrainTensor = torch.LongTensor(y_train)
yTestTensor = torch.LongTensor(y_test)


<ipython-input-56-ece8ed848272>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  XTrainTensor = torch.FloatTensor(X_train)


In [57]:
input_size = XTrainTensor.shape[1]
hidden1_size = 50
hidden2_size = 5
output_size = 2

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')

model = nn.Sequential(
    nn.Linear(input_size, hidden1_size),
    nn.ReLU(),
     nn.BatchNorm1d(hidden1_size),
    nn.Dropout(0.4),
    nn.Linear(hidden1_size, hidden2_size),
    nn.ReLU(),
    nn.BatchNorm1d(hidden2_size),
    nn.Linear(hidden2_size, output_size),
    # nn.Softmax(dim=1)
)

model.apply(init_weights)

# Loss function and optimizer; this was the best results after comparing with optimiser like sgd, and other loss functions
criterion = nn.CrossEntropyLoss()
# optimizer= torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.01)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(XTrainTensor)
    loss = criterion(outputs, yTrainTensor)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}]')


with torch.no_grad():
    model.eval()
    outputs = model(XTestTensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test, predicted.numpy())

print(f'Accuracy on Testing Split: {accuracy * 100:.2f}%')


print("Classification Report:")
print(classification_report(y_test, predicted.numpy()))

Epoch [10/500]
Epoch [20/500]
Epoch [30/500]
Epoch [40/500]
Epoch [50/500]
Epoch [60/500]
Epoch [70/500]
Epoch [80/500]
Epoch [90/500]
Epoch [100/500]
Epoch [110/500]
Epoch [120/500]
Epoch [130/500]
Epoch [140/500]
Epoch [150/500]
Epoch [160/500]
Epoch [170/500]
Epoch [180/500]
Epoch [190/500]
Epoch [200/500]
Epoch [210/500]
Epoch [220/500]
Epoch [230/500]
Epoch [240/500]
Epoch [250/500]
Epoch [260/500]
Epoch [270/500]
Epoch [280/500]
Epoch [290/500]
Epoch [300/500]
Epoch [310/500]
Epoch [320/500]
Epoch [330/500]
Epoch [340/500]
Epoch [350/500]
Epoch [360/500]
Epoch [370/500]
Epoch [380/500]
Epoch [390/500]
Epoch [400/500]
Epoch [410/500]
Epoch [420/500]
Epoch [430/500]
Epoch [440/500]
Epoch [450/500]
Epoch [460/500]
Epoch [470/500]
Epoch [480/500]
Epoch [490/500]
Epoch [500/500]
Accuracy on Testing Split: 83.97%
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.85      0.84      9965
           1       0.85      0.83      0.84

4b

In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.utils import shuffle

In [58]:
input_size = 3000  # Assuming 300-dimensional word vectors(as prtrained is of 300) and considering the first ten words
hidden_size = 50
output_size = 2

In [11]:
w2v = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


start4b

In [61]:
max_review_length = 10
X_word2vec_mlp = [
    [w2v[word] if word in w2v else np.zeros(300) for word in review.split()][:max_review_length]
    for review in sdf['review_body']
]

X_word2vec_mlp = [
    vec + [np.zeros(300)] * (max_review_length - len(vec)) for vec in X_word2vec_mlp
]


X_word2vec_ffn = torch.FloatTensor(X_word2vec_mlp)
y_ffn = torch.FloatTensor(sdf['sentiment'].astype(int))  # for BCE loss converted to float


X_word2vec_ffn, y_ffn = shuffle(X_word2vec_ffn, y_ffn, random_state=42)
X_train_ffn, X_test_ffn, y_train_ffn, y_test_ffn = train_test_split(X_word2vec_ffn, y_ffn, test_size=0.2, random_state=42)


In [62]:
input_size_ffn = 3000  # 300 features for each of the first 10 words
hidden_size1_ffn = 50
hidden_size2_ffn = 5
output_size_ffn = 1  # Single output neuron for binary classification
batch_size_ffn = 32

modelFFN = nn.Sequential(
    nn.Linear(input_size_ffn, hidden_size1_ffn),
    nn.ReLU(),
    nn.Linear(hidden_size1_ffn, hidden_size2_ffn),
    nn.ReLU(),
    nn.Linear(hidden_size2_ffn, output_size_ffn)
)

In [63]:
criterion_ffn = nn.BCEWithLogitsLoss()
optimizer_ffn = optim.Adam(modelFFN.parameters(), lr=0.001)
scheduler_ffn = optim.lr_scheduler.StepLR(optimizer_ffn, step_size=5, gamma=0.5)

train_dataset_ffn = TensorDataset(X_train_ffn.view(-1, input_size_ffn), y_train_ffn.view(-1, 1))  # Reshape X and y
train_loader_ffn = DataLoader(train_dataset_ffn, batch_size=batch_size_ffn, shuffle=True)


In [64]:
numEpochsFFN = 20
for epoch in range(numEpochsFFN):
    modelFFN.train()
    for batch_X, batch_y in train_loader_ffn:
        optimizer_ffn.zero_grad()
        outputs_ffn = modelFFN(batch_X.view(-1, input_size_ffn))
        loss_ffn = criterion_ffn(outputs_ffn, batch_y)
        loss_ffn.backward()
        optimizer_ffn.step()
    scheduler_ffn.step()

    if (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{numEpochsFFN}]')


modelFFN.eval()
with torch.no_grad():
    outputs_ffn = modelFFN(X_test_ffn.view(-1, input_size_ffn))  # Reshape input
    predicted_ffn = (torch.sigmoid(outputs_ffn) > 0.5).float()  # Convert to binary predictions
    accuracy_ffn = accuracy_score(y_test_ffn, predicted_ffn.numpy())

print(f'Accuracy on Testing Split with FFN: {accuracy_ffn * 100:.2f}%')

Epoch [2/20]
Epoch [4/20]
Epoch [6/20]
Epoch [8/20]
Epoch [10/20]
Epoch [12/20]
Epoch [14/20]
Epoch [16/20]
Epoch [18/20]
Epoch [20/20]
Accuracy on Testing Split with FFN: 73.39%


end4b

Simpler models could be more robust in smaller dataset, as they are less likely to memorize noise in the training data. Hence FFN with aveage vector(4a) is comparable, but the concatenated vector (4b) has a lesser accuracy

5


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import gensim.downloader as api
import numpy as np

5a) Using Recurrent Neural Networks to predict the sentiment for the test case.

The vectors are got by considering first ten words of the review, and padding with zero if the size is less than 10

In [12]:
max_review_length = 10
X_word2vec_rnn = [
    [w2v[word] if word in w2v else np.zeros(300) for word in review.split()][:max_review_length]
    for review in sdf['review_body']
]

X_word2vec_rnn = [
    vec + [np.zeros(300)] * (max_review_length - len(vec)) for vec in X_word2vec_rnn
]

X_word2vec_rnn = torch.FloatTensor(X_word2vec_rnn)
y_rnn = torch.FloatTensor(sdf['sentiment'].astype(int))


<ipython-input-12-076ed7908424>:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X_word2vec_rnn = torch.FloatTensor(X_word2vec_rnn)


In [13]:
X_word2vec_rnn, y_rnn = shuffle(X_word2vec_rnn, y_rnn, random_state=42)
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X_word2vec_rnn, y_rnn, test_size=0.2, random_state=42)


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, e = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

input_size_rnn = 300
hidden_size_rnn = 10
output_size_rnn = 1
batch_size_rnn = 32


In [14]:
model_rnn = RNN(input_size_rnn, hidden_size_rnn, output_size_rnn)
criterion_rnn = nn.BCEWithLogitsLoss()
optimizer_rnn = optim.Adam(model_rnn.parameters(), lr=0.001)
scheduler_rnn = optim.lr_scheduler.StepLR(optimizer_rnn, step_size=5, gamma=0.5)


train_dataset_rnn = TensorDataset(X_train_rnn, y_train_rnn.view(-1, 1))
train_loader_rnn = DataLoader(train_dataset_rnn, batch_size=batch_size_rnn, shuffle=True)


In [18]:
num_epochs_rnn = 20
for epoch in range(num_epochs_rnn):
    model_rnn.train()
    for batch_X, batch_y in train_loader_rnn:
        # print(batch_X.shape)
        optimizer_rnn.zero_grad()
        outputs_rnn = model_rnn(batch_X)
        loss_rnn = criterion_rnn(outputs_rnn, batch_y)
        loss_rnn.backward()
        torch.nn.utils.clip_grad_norm_(model_rnn.parameters(), max_norm=1) #to overcome exploding gradient problem
        optimizer_rnn.step()
    scheduler_rnn.step()
    if (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs_rnn}]')

model_rnn.eval()
with torch.no_grad():
    outputs_rnn = model_rnn(X_test_rnn)
    predicted_rnn = (torch.sigmoid(outputs_rnn) > 0.5).float()
    accuracy_rnn = accuracy_score(y_test_rnn, predicted_rnn.numpy())

print(f'Accuracy on Testing Split with RNN: {accuracy_rnn * 100:.2f}%')

Epoch [2/20]
Epoch [4/20]
Epoch [6/20]
Epoch [8/20]
Epoch [10/20]
Epoch [12/20]
Epoch [14/20]
Epoch [16/20]
Epoch [18/20]
Epoch [20/20]
Accuracy on Testing Split with RNN: 77.62%


5b starts

5b) Using LSTM to predict the sentiment for the test case.

The vectors are got by considering first ten words of the review, and padding with zero if the size is less than 10

because GPU utilisation is very high, i have created vectors seperately for each of the three parts in this sections

In [15]:
max_review_length = 10
X_word2vec_lstm = [
    [w2v[word] if word in w2v else np.zeros(300) for word in review.split()][:max_review_length]
    for review in sdf['review_body']
]

X_word2vec_lstm = [
    vec + [np.zeros(300)] * (max_review_length - len(vec)) for vec in X_word2vec_lstm
]

X_word2vec_lstm = torch.FloatTensor(X_word2vec_lstm)
y_lstm = torch.FloatTensor(sdf['sentiment'].astype(int))

X_word2vec_lstm, y_lstm = shuffle(X_word2vec_lstm, y_lstm, random_state=42)
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_word2vec_lstm, y_lstm, test_size=0.2, random_state=42)




<ipython-input-15-1bc9038752db>:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X_word2vec_lstm = torch.FloatTensor(X_word2vec_lstm)


In [16]:

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

input_size_lstm = 300
hidden_size_lstm = 10
output_size_lstm = 1
batch_size_lstm = 32


In [17]:
model_lstm = LSTMModel(input_size_lstm, hidden_size_lstm, output_size_lstm)
criterion_lstm = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer_lstm = optim.Adam(model_lstm.parameters(), lr=0.001)
scheduler_lstm = optim.lr_scheduler.StepLR(optimizer_lstm, step_size=5, gamma=0.5)

train_dataset_lstm = TensorDataset(X_train_lstm, y_train_lstm.view(-1, 1))
train_loader_lstm = DataLoader(train_dataset_lstm, batch_size=batch_size_lstm, shuffle=True)


num_epochs_lstm = 20
for epoch in range(num_epochs_lstm):
    model_lstm.train()
    for batch_X, batch_y in train_loader_lstm:
        optimizer_lstm.zero_grad()
        outputs_lstm = model_lstm(batch_X)
        loss_lstm = criterion_lstm(outputs_lstm, batch_y)
        loss_lstm.backward()
        optimizer_lstm.step()
    scheduler_lstm.step()

    if (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs_lstm}]')


model_lstm.eval()
with torch.no_grad():
    outputs_lstm = model_lstm(X_test_lstm)
    predicted_lstm = (torch.sigmoid(outputs_lstm) > 0.5).float()
    accuracy_lstm = accuracy_score(y_test_lstm, predicted_lstm.numpy())

print(f'Accuracy on Testing Split with LSTM: {accuracy_lstm * 100:.2f}%')

Epoch [2/20]
Epoch [4/20]
Epoch [6/20]
Epoch [8/20]
Epoch [10/20]
Epoch [12/20]
Epoch [14/20]
Epoch [16/20]
Epoch [18/20]
Epoch [20/20]
Accuracy on Testing Split with LSTM: 79.50%


5c

5c) Using Gated Recurrent Units(GRU Cell) to predict the sentiment for the test case.

The vectors are got by considering first ten words of the review, and padding with zero if the size is less than 10

In [13]:


X_word2vec_gru = [
    [w2v[word] if word in w2v else np.zeros(300) for word in review.split()][:10]
    for review in sdf['review_body']
]

X_word2vec_gru = [
    vec + [np.zeros(300)] * (max_review_length - len(vec)) for vec in X_word2vec_gru
]


X_word2vec_gru = torch.FloatTensor(X_word2vec_gru)
y_gru = torch.FloatTensor(sdf['sentiment'].astype(int))

X_word2vec_gru, y_gru = shuffle(X_word2vec_gru, y_gru, random_state=42)
X_train_gru, X_test_gru, y_train_gru, y_test_gru = train_test_split(X_word2vec_gru, y_gru, test_size=0.2, random_state=42)

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])
        return out

input_size_gru = 300
hidden_size_gru = 10
output_size_gru = 1
batch_size_gru = 32

model_gru = GRUModel(input_size_gru, hidden_size_gru, output_size_gru)
criterion_gru = nn.BCEWithLogitsLoss()
optimizer_gru = optim.Adam(model_gru.parameters(), lr=0.001)
scheduler_gru = optim.lr_scheduler.StepLR(optimizer_gru, step_size=5, gamma=0.5)


train_dataset_gru = TensorDataset(X_train_gru, y_train_gru.view(-1, 1))  # Reshape y
train_loader_gru = DataLoader(train_dataset_gru, batch_size=batch_size_gru, shuffle=True)

# Training loop
num_epochs_gru = 100
for epoch in range(num_epochs_gru):
    model_gru.train()
    for batch_X, batch_y in train_loader_gru:
        optimizer_gru.zero_grad()
        outputs_gru = model_gru(batch_X)
        loss_gru = criterion_gru(outputs_gru, batch_y)
        loss_gru.backward()
        optimizer_gru.step()
    scheduler_gru.step()

    if (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs_gru}]')

# Evaluation on the testing split
model_gru.eval()
with torch.no_grad():
    outputs_gru = model_gru(X_test_gru)
    predicted_gru = (torch.sigmoid(outputs_gru) > 0.5).float()
    accuracy_gru = accuracy_score(y_test_gru, predicted_gru.numpy())

print(f'Accuracy on Testing Split with GRU: {accuracy_gru * 100:.2f}%')


<ipython-input-13-f3618bb8bab1>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  X_word2vec_gru = torch.FloatTensor(X_word2vec_gru)


Epoch [2/20], Loss: 0.3779
Epoch [4/20], Loss: 0.4204
Epoch [6/20], Loss: 0.6012
Epoch [8/20], Loss: 0.3501
Epoch [10/20], Loss: 0.4237
Epoch [12/20], Loss: 0.6609
Epoch [14/20], Loss: 0.5397
Epoch [16/20], Loss: 0.4133
Epoch [18/20], Loss: 0.3039
Epoch [20/20], Loss: 0.2272
Accuracy on Testing Split with GRU: 79.58%


We can observe that for the RNN which considers a vector created by truncated review with 10 words and padding the smaller ones, accuracies are better for RNN compared to FFN; this suggests that the sequential relationship is high, hence temporal context results in a higher accuracy

The accuracies for the various models can be summarised as follows:

1. Perceptron : 67
; Perceptron (TFIDF) : 82
3. SVM : 81 ; SVM (TFIDF) : 86
5. Feed forward Network (Average review vector) : 83.39
6. Feed forward Network (Concatenated 10 vectors) : 73.37
7. RNN : 77.62
8. LSTM Cell : 79.50
9. GRU Cell : 79.58



